In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-534c92b1-d0c4-521f-c011-5c5bb20afca4)


In [2]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
import tensorflow as tf
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
def preprocess_features(images):
    # Convert pixel values from 0-255 to 0-1
    images = images.astype('float32')/255
    return images


def one_hot_label(labels, num_classes=10, log=False):
    labels = keras.utils.to_categorical(labels, num_classes)

    if log == True:
	    print('One-hot encoded labels.')
	    print('Example image label', labels[0])
    return labels

In [0]:
def load_dataset():
    print('Loading MNIST dataset')
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    print('Shape of image: ', x_train[0].shape)
    return x_train, y_train, x_test, y_test


def extract(display_images=False, reshape_input=False, one_hot=True):
    x_train, y_train, x_test, y_test = load_dataset()

    #Preprocess features
    x_train = preprocess_features(x_train)
    x_test = preprocess_features(x_test)

    #Preprocess label
    if one_hot:
        y_train = one_hot_label(y_train, log=True)
        y_test = one_hot_label(y_test)

    if reshape_input:
        x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
        x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    return x_train, y_train,  x_test, y_test

In [0]:
def build_model():
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(filters=4,
                    kernel_size=2,
                    padding='same', 
                    activation='relu',
                    input_shape=(28, 28, 1)))
    model.add(Conv2D(filters=4,
                    kernel_size=2,
                    padding='same',
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=4))

    # Deep Neural Network
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.summary()

    return model

In [7]:
x_train, y_train, x_test, y_test = extract(reshape_input=True)

# Build and compile model
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit data for training set
print('fitting')
hist = model.fit(x_train, y_train, batch_size=32, epochs=1,
      verbose=1, shuffle=True)

score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])

Loading MNIST dataset
11493376/11490434 [==============================] - 2s 0us/step
Shape of image:  (28, 28)
One-hot encoded labels.
Example image label [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 4)         20        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 4)         68        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 4)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 196)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25216     
____________